In [1]:
#import important libraries.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.api.types import is_numeric_dtype
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error, accuracy_score
import math
import warnings
warnings.filterwarnings('ignore')

In [2]:
#read dataset csv file.
df = pd.read_csv("Churn_Modelling.csv")

In [3]:
# find the features and observation in the data set.
df.shape

(10000, 14)

In [4]:
# find the datatypes of the features and target variable. 
df.dtypes

RowNumber            int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [5]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
dropped = ["RowNumber", "CustomerId", "Surname"]
for i in range(len(dropped)):
    del df[dropped[i]]
df.head()    

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
df["Balance"] = np.ceil(df["Balance"]).astype(int)
df["EstimatedSalary"] = np.ceil(df["EstimatedSalary"]).astype(int)

In [8]:
df = pd.get_dummies(df)
df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0,1,1,1,101349,1,1,0,0,1,0
1,608,41,1,83808,1,0,1,112543,0,0,0,1,1,0
2,502,42,8,159661,3,1,0,113932,1,1,0,0,1,0
3,699,39,1,0,2,0,0,93827,0,1,0,0,1,0
4,850,43,2,125511,1,1,1,79085,0,0,0,1,1,0


In [9]:
def remove_outlier(feature):
            first_q = np.percentile(df[feature], 25)
            third_q = np.percentile(df[feature], 75)
            IQR = third_q - first_q
            IQR *= 1.5
            minimum = first_q - IQR
            maximum = third_q + IQR

            mean = df[feature].median()

            df.loc[df[feature] < minimum, feature] = mean
            df.loc[df[feature] > maximum, feature] = mean

outliers = ["CreditScore", "Age", "NumOfProducts"]
for i in range(len(outliers)):
    remove_outlier(outliers[i])
    

In [10]:
df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619.0,42.0,2,0,1.0,1,1,101349,1,1,0,0,1,0
1,608.0,41.0,1,83808,1.0,0,1,112543,0,0,0,1,1,0
2,502.0,42.0,8,159661,3.0,1,0,113932,1,1,0,0,1,0
3,699.0,39.0,1,0,2.0,0,0,93827,0,1,0,0,1,0
4,850.0,43.0,2,125511,1.0,1,1,79085,0,0,0,1,1,0


In [11]:
X = df.iloc[:, df.columns != "Exited"]
y = df["Exited"]

In [12]:
#Using Python Code
class LinearTrainer:

    def __init__(self):

        # Learning Rate
        self.l_rate = 0.0004

        # Total iterations
        self.iterations = 2000

    def trains(self, x_data_train, y_data_train):

        iterative_value = 0
        x_data_train = np.transpose(x_data_train)
        y_data_train = np.transpose(y_data_train)
        data_set_size = x_data_train.shape[1]

        # weights_value matrix shape = (data_set_size x 1)
        weights_value = np.random.rand(13, 1) * 0.001

        # biased_value matrix shape = (1 x 1)
        biased_value = np.zeros((1, 1), dtype='float')

        while iterative_value < self.iterations:

            # z = w.T*x + b and z matrix shape = (1 x data_set_size)
            z = np.dot(weights_value.T, x_data_train) + biased_value

            # y^_train = sigmoid(z) and y^ matrix shape = (1 x data_set_size)
            y_data_train_predicated = 1 / 1 + np.exp(-z)

            # dz = y^_train - y and dz matrix shape = (1 x data_set_size)
            dz = y_data_train_predicated - y_data_train

            # dw = (x * dz.T ) / data_set_size and dw matrix shape = (features_size x 1)
            dw = np.dot(x_data_train, dz.T) / data_set_size

            # db = (dz.sum ) / data_set_size and db matrix shape = (1 x 1)
            db = dz.sum() / data_set_size

            # w = w - alpha * dw and w matrix shape = (features_size x 1)
            weights_value = weights_value - np.dot(self.l_rate, dw)

            # b = b - alpha * db and b matrix shape = (features_size x 1)
            biased_value = biased_value - np.dot(self.l_rate, db)

            iterative_value += 1

        return weights_value, biased_value

    def classify(self, x_data_test, parameters):

        # z = w.T*x + b and z matrix shape = (1 x data_set_size)
        z = np.dot(parameters[0].T, x_data_test.T) + parameters[1]

        # y^_test = sigmoid(z) and y^ matrix shape = (1 x data_set_size)
        y_data_test_predicted = 1 / 1 + np.exp(-z)

        return y_data_test_predicted

    def accuracy(self, y_data_test, y_pred_test):

        y_pred_test = np.nan_to_num(y_pred_test)
        y_pred_test = y_pred_test.T
        accuracy = mean_absolute_error(y_data_test, y_pred_test)
        return (1 - accuracy) * 100

    def plotgraph(self,  x_data_test, y_data_test, y_pred):
        plt.plot(x_data_test, y_data_test, 'or', label='whole data')
        plt.plot(x_data_test, y_pred, label='predicted value')
        plt.legend()
        plt.show()

def main():
    l_t = LinearTrainer()    
    x_data = np.array(X)
    y_data = np.array(y)
    y_data = y_data.reshape(len(y_data), 1)

    x_data_train, x_data_test, y_data_train, y_data_test = train_test_split(x_data, y_data
                                                                            , test_size=0.3, random_state=42)

    l_t = LinearTrainer()
    parameters = l_t.trains(x_data_train, y_data_train)
    y_prediction = l_t.classify(x_data_test, parameters)
    y_prediction_train = l_t.classify(x_data_train, parameters)
    accuracy = l_t.accuracy(y_data_test, y_prediction)
    accuracy_train = l_t.accuracy(y_data_train, y_prediction_train)

    print("Train accuracy: ", accuracy_train)
    print("Test accuracy: ", accuracy)


if __name__ == '__main__':
    main()



Train accuracy:  79.24285714285715
Test accuracy:  80.53333333333333


In [13]:
# Using sklearn linear regression model
x_data_train, x_data_test, y_data_train, y_data_test = train_test_split(X, y, test_size=0.3, random_state=0)

model = MLPClassifier(solver='lbfgs', alpha=1e-5, random_state=1)
reg = model.fit(x_data_train,y_data_train)
reg.score(x_data_test, y_data_test)
y_pred =  reg.predict(x_data_test)
print("Train accuracy", accuracy_score(y_data_train, y_data_train) * 100)
print("Test accuracy", accuracy_score(y_data_test, y_pred) * 100)

Train accuracy 100.0
Test accuracy 79.13333333333334
